In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !cp -r /content/drive/MyDrive/calcification-detection-project/calcification_detecion/new_data_rois/data_rois.zip /home/
# !unzip /home/data_rois.zip -d /home
# !mv /home/home/vzalevskyi/projects/data_rois /home/data_rois
# !rm -r /home/home

In [3]:
from pathlib import Path
import os
# Vlad's drive
repo_path = Path.cwd()/'drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/notebooks'
# Joaquin's drive
# repo_path = Path.cwd()/'drive/MyDrive/calcification_detection/calc-det/notebooks'
os.chdir(str(repo_path))

thispath = Path.cwd().resolve()
import sys; sys.path.insert(0, str(thispath.parent))

In [4]:
from deep_learning.dataset.dataset import INBreast_Dataset_pytorch

import copy
import torch
import time
import random
import pickle
from general_utils.plots import simple_im_show, simple_im_show2
from collections import Counter

from metrics.metrics import froc_curve
from metrics.metrics_utils import get_froc_df_of_img, get_froc_df_of_many_imgs_features, get_tp_fp_fn_center_patch_criteria

from deep_learning.dataset.dataset import ImgCropsDataset
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T
import torch
from torch.optim import lr_scheduler
from torchvision import models
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import pandas as pd

from database.dataset import INBreast_Dataset
import torchvision.transforms.functional as F
from torchvision.utils import draw_bounding_boxes
from deep_learning.models.base_classifier import CNNClasssifier
from deep_learning.models.detect_based_detector import DetectionBasedDetector
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from general_utils.plots import plot_froc
from general_utils.utils import get_center_bboxes
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator


In [5]:
dataset_arguments = {
        'extract_patches': False,
        'delete_previous': False,
        'extract_patches_method': 'all',
        'patch_size': 224,
        'stride': 100,
        'min_breast_fraction_roi': 0.7,
        'n_jobs': -1,
        'cropped_imgs': True,
        'ignore_diameter_px': 15,
        'train_neg_to_pos_ratio': None,
        'balancing_seed': 0,
        'normalization': 'min_max',
        'get_lesion_bboxes': True,
        'for_detection_net': True,
        'patch_images_path': Path('/home/data_rois/'),
        'detection_bbox_size':14}

image_datasets = {
    'train': INBreast_Dataset_pytorch(
        partitions=['train'],
        **dataset_arguments
    ),
    'val': INBreast_Dataset_pytorch(
        partitions=['validation'],

        **dataset_arguments
    ),
    
}

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:52: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=get_lesion_bboxes, max_lesion_diam_mm=None, use_muscle_mask=False


In [8]:
chkp_path = "/content/drive/MyDrive/calcification-detection-project/calcification_detecion/data/deepl_runs/dns121ptr0/dns121ptr0_AP_IoU_0.50_all.pt"
chkp_model = torch.load(chkp_path)
det = DetectionBasedDetector(chkp_model, True, 224, 200, None, 8, 14, device, True, iou_threshold=0.5)


In [9]:
list(p for p in det.model.parameters() if p.requires_grad)

[]

Plotting some results

In [10]:
# for i in range(3):

#     timage, ttarget = image_datasets['val'][i]
#     with torch.no_grad():
#         pred_target = model([timage.to(device)])[0]


#     result = draw_bounding_boxes((255*timage).to(torch.uint8), ttarget['boxes'], colors=['blue']*len(ttarget['boxes']),  width=1)
#     show(result)
#     result = draw_bounding_boxes((255*timage).to(torch.uint8), pred_target['boxes'], colors=['red']*len(pred_target['boxes']),  width=1)
#     show(result)

#     print(ttarget['boxes'])
#     print(pred_target['boxes'])
    
#     plt.show()

```AttributeError: 'NoneType' object has no attribute 'min'``` might indicate that you dont have the patches in the path provided to the database classes at `patch_images_path`

In [11]:
full_imgs_db = INBreast_Dataset(
    return_lesions_mask=True,
    level='image',
    max_lesion_diam_mm=None,
    extract_patches=False,
    partitions=['validation'],
    lesion_types=['calcification', 'cluster'],
    cropped_imgs=True,
    keep_just_images_of_lesion_type=False,
    use_muscle_mask=False,
    ignore_diameter_px=15
)

In [12]:
tps = []
fps = []
fns = []
ignoreds = []

R = 8 # radius of the patch (for now hardcoded since we've used 14x14 patches for training)


for idx in tqdm(range(len(full_imgs_db))):
    img = full_imgs_db[idx]['img']
    lesion_bboxes = full_imgs_db[idx]['lesion_bboxes']
    img_id = full_imgs_db[idx]['img_id']
    lesion_mask = full_imgs_db[idx]['lesion_mask']

    cands = det.detect(img)
    
    # convert predictions from [x1, x2, y1, y2] to (x_center, y_center, radius) convention
    # used by labelling
    cands_xyr = get_center_bboxes([((x[0], x[2]), (x[1], x[3])) for x in cands])
    
    # add radius column
    cands_xyr = np.append(cands_xyr, R*np.ones((len(cands_xyr), 1)), axis=1)

    # add score column
    cands_xyr = np.append(cands_xyr, cands[:,4].reshape(-1, 1), axis=1)

    tp, fp, fn, ignored = get_tp_fp_fn_center_patch_criteria(cands_xyr, lesion_mask, None, 14, True, scores_passed=True)
    
    tp['img_id'] = img_id
    fp['img_id'] = img_id
    fn['img_id'] = img_id
    ignored['img_id'] = img_id
    tps.append(tp)
    fps.append(fp)
    fns.append(fn)
    ignoreds.append(ignored)

  0%|          | 0/62 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
  5%|▍         | 3/62 [00:30<10:01, 10.20s/it]


KeyboardInterrupt: ignored

In [41]:
test_features = pd.concat(tps + fps)
fns_df = pd.concat(fns)
ignoreds_df = pd.concat(ignoreds)

In [42]:
test_features['candidate_coordinates'] = [[r['x'] , r['y'] , r['radius']] for idx, r in test_features.iterrows()]
test_features['label'] = test_features.label == 'TP'
test_features.head()

,x,y,radius,score,label,repeted_idxs,matching_gt,img_id,candidate_coordinates
0,1515.0,1551.0,8.0,0.486264,True,0,1,20587294,"[1515.0, 1551.0, 8.0]"
1,988.0,2653.0,8.0,0.481735,True,1,2,20587294,"[988.0, 2653.0, 8.0]"
0,542.0,226.0,8.0,0.495512,True,0,5,20587320,"[542.0, 226.0, 8.0]"
1,533.0,217.0,8.0,0.473210,True,1,4,20587320,"[533.0, 217.0, 8.0]"
2,533.0,226.0,8.0,0.471606,True,2,4,20587320,"[533.0, 226.0, 8.0]"


In [ ]:
normal_imgs_ids = full_imgs_db.get_normal_imgs_ids()


froc_df_1st = get_froc_df_of_many_imgs_features(
    test_features[['candidate_coordinates', 'label', 'img_id', 'matching_gt', 'repeted_idxs']],
    fns_df,
    test_features.score.values,
    normal_imgs_ids
)

sens1, avgs_fp_per_image1, _ = froc_curve(froc_df_1st, non_max_supression=True, cut_on_50fpi=True)


In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(15, 10))
plot_froc(avgs_fp_per_image1, sens1, title='FROC DL', cut_on_50fpi=True, ax=axs)

